In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('MF.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 150 entries, Cate_name to Yield
dtypes: int64(148), object(2)
memory usage: 79.8+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MF_9  Cate_MF_33  Cate_MF_58  Cate_MF_68  Cate_MF_80  Cate_MF_102  \
0          0           0           0           0           0            0   
1          0           0           0           0           0            0   
2          0           0           0           0           0            0   
3          0           0           0           0           0            0   
4          0           0           0           0           0            0   

   Cate_MF_105  Cate_MF_114  Cate_MF_125  Cate_MF_142  ...  P_MF_1154  \
0            0            0            0            0  ...          1   
1            0            0            0            0  ...          0   
2            0            0            0            0  ...          0   
3            0            0            0            0  ...          0   
4            0            0            0            1  ...          1   

   P_MF_1199  P_MF_1272  P_MF_1380  P_MF_1570  P_MF_1689  P_MF_1750  \
0        

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []
loo_score = []

#knn
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train = a_X_train.dropna(how='any', axis=1)
    a_X_test = a_X_test[a_X_train.columns]
    
    param_knn={'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'p':[1,2,3,4,5]}
    reg_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_knn, cv=5, n_jobs=2)
    reg_knn.fit(a_X_train,y_train['Yield'])
    reg_best = reg_knn.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)
    y_pred2 = reg_best.predict(a_X_test)

    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    parametors = reg_knn.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_knn.best_score_
    data_cv_score.append(cv_score)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_knn.best_params_)
    print("Best Model Score:",reg_knn.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.6207874198931329
R2_test: 0.46400335063551124
----------------------
seed: 1
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.5089280099498696
R2_test: 0.624459598894489
----------------------
seed: 2
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.5620514894861026
R2_test: 0.5808715038900378
----------------------
seed: 3
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.4638270397267544
R2_test: 0.586756685796507
----------------------
seed: 4
Best Model Parameter: {'n_neighbors': 9, 'p': 1}
Best Model Score: 0.5175667062817579
R2_test: 0.6092340874758571
----------------------
seed: 5
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.5726420163727071
R2_test: 0.5875287860554834
----------------------
seed: 6
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.6069833285131058
R2_test: 0.650

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/MF/knn_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.706070   14.720886  12.198582  0.464003  26.477479  21.087302   
1     0.666942   17.354757  13.911854  0.624460  19.074632  14.931973   
2     0.678600   17.243157  13.771277  0.580872  20.132633  17.940476   
3     0.665481   17.663795  13.789894  0.586757  19.915035  16.559524   
4     0.622925   18.029847  13.858156  0.609234  20.384084  16.095238   
5     0.644200   16.973658  13.677812  0.587529  22.397526  16.333333   
6     0.708455   15.468149  12.353723  0.650545  20.507439  13.404762   
7     0.681024   17.467108  13.942249  0.530510  20.629995  17.217687   
8     0.756797   14.630352  10.994681  0.600689  19.864890  16.303571   
9     0.623489   19.533888  15.982979  0.688434  14.817139  12.733333   
mean  0.675398   16.908560  13.448121  0.592303  20.420085  16.260720   
std   0.037286    1.394737   1.210006  0.055993   2.602524   2.121782   

      cv_score  
0     0.620787  
1     0.508928  